## Chapter 10

### Exercise 1

In [1]:
from utils import set_mode

set_mode('local')

In [2]:
import time
from dsets import LunaDataset

def time_iterations(iter_count):
    ds = LunaDataset()

    start = time.time()
    for i in range(iter_count):
        var = ds[i]
    end = time.time()

    print(f'{iter_count} iterations finished in {end - start} seconds.')

def time_last_iterations(iter_count):
    ds = LunaDataset()

    start = time.time()
    for i in range(iter_count):
        var = ds[-i]
    end = time.time()

    print(f'{iter_count} iterations finished in {end - start} seconds.')

In [3]:
time_iterations(1000)

2025-07-27 09:55:36,716 INFO     pid:11272 dsets:175:__init__ <dsets.LunaDataset object at 0x7f6829b2adb0>: 551065 training samples


1000 iterations finished in 291.07224130630493 seconds.


a) First run finished in 141 seconds.

In [5]:
time_iterations(1000)

1000 iterations finished in 0.7033522129058838 seconds.


b) Second run finished in under 1 second.

In [6]:
time_iterations(1000)

1000 iterations finished in 208.80103278160095 seconds.


c) After clearing the cache, the runtime is back to 200 seconds.

In [8]:
time_last_iterations(1000)
time_last_iterations(1000)

1000 iterations finished in 2.2063560485839844 seconds.
1000 iterations finished in 0.22845101356506348 seconds.


d) Using the last 1000 samples has no impact on the runtime after being cached.

### Exercise 2

In [2]:
time_iterations(1000)
time_iterations(1000)

1000 iterations finished in 225.06759643554688 seconds.
1000 iterations finished in 211.10741472244263 seconds.


After randomizing the list, both runs take quite a long time.

### Exercise 3

In [2]:
time_iterations(1000)
time_iterations(1000)

New instance of LunaDataset created.
1000 iterations finished in 278.02917885780334 seconds.
New instance of LunaDataset created.
1000 iterations finished in 0.6841669082641602 seconds.


The getCt decorator does have an impact on the first loop. The second one remains the same, however.

## Chapter 11

In [ ]:
#TODO

## Chapter 12

### Exercise 1

In [ ]:
import numpy as np

In [ ]:
# More general implementation of f_score: Recall is considered beta times as important as precision
def f_score(preds, labels, beta=1, classification_threshold=0.5):
    # True positives: Elements identified as nodules that are actually nodules
    # False positives: Elements identified as nodules that are not nodules
    # True negatives: Elements not identified as nodules that are not nodules
    # False negatives: Elements not identified as nodules that are nodules

    pos_label_mask = labels > classification_threshold  # Actual nodules
    pos_pred_mask = preds > classification_threshold    # Elements identified as nodules

    neg_label_mask = ~pos_label_mask    # Actual non-nodules
    neg_pred_mask = ~pos_pred_mask      # Elements identifies as non-nodules

    pos_count = int(pos_label_mask.sum())   # Number of actual nodules
    neg_count = int(neg_label_mask.sum())   # Number of actual non-nodules

    true_neg_count = int((neg_label_mask & neg_pred_mask).sum())    # Number of non-nodules identified as such
    true_pos_count = int((pos_label_mask & pos_pred_mask).sum())    # Number of nodules identified as such

    false_pos_count = neg_count - true_neg_count    # Num. of samples identified as nodules, even though they are not nodules
    false_neg_count = pos_count - true_pos_count    # Num. of samples identified as non-nodules, even though they are nodules

    pos_pred_count = np.float32(true_pos_count + false_pos_count)
    precision = true_pos_count / pos_pred_count if pos_pred_count > 0 else 0

    act_pos_count = np.float32(true_pos_count + false_neg_count)
    recall = true_pos_count / act_pos_count if act_pos_count > 0 else 0

    denominator = ((beta ** 2) * precision) + recall
    return (1 + beta ** 2) * (precision * recall) / denominator if denominator > 0 else 0.0

b)
To reiterate:

- Recall: Number of samples correctly identified as positive against the number of actual positive samples
- Precision: Number of samples correctly identified as positive against the number of samples identified as positive, whether wrong or right

If we classify everything as positive, we have a lot of false positives, so precision will be very low. We will not have any false negatives, however, so recall will be high. If we classify everything as negative, both will be 0. In our case, we want to minimize false negatives, as we want to be really sure we miss no nodules, so we want to weigh recall higher than precision. In this case the F2 score is a better choice.

In [ ]:
# TODO: Finish other exercises